In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

df = pd.read_csv("../data/train_5folds.csv")
df.head()

,Unnamed: 0,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,kfold
0,0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,0
1,1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,2
2,2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,3
3,3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,3
4,4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,1


In [3]:
from pathlib import Path

train_image_fpaths = [Path(f"../data/train/{i}.jpg") for i in df["Id"].values.tolist()]
train_image_fpaths[:5]

[PosixPath('../data/train/0007de18844b0dbbb5e1f607da0606e0.jpg'),
 PosixPath('../data/train/0009c66b9439883ba2750fb825e1d7db.jpg'),
 PosixPath('../data/train/0013fd999caf9a3efe1352ca1b0d937e.jpg'),
 PosixPath('../data/train/0018df346ac9c1d8413cfcc888ca8246.jpg'),
 PosixPath('../data/train/001dc955e10590d3ca4673f034feeef2.jpg')]

In [4]:
train_targets = [[1 if t >= 80 else 0] for t in df.Pawpularity.values.tolist()]
train_targets[:20]

[[0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [1],
 [0],
 [0],
 [0],
 [1]]

In [5]:
from ml.vision.data import ImageDataModule
from timm.data import transforms_factory

In [6]:
train_aug = transforms_factory.create_transform(
    input_size=224,
    is_training=True,
    auto_augment='rand-n2-m10',
)
val_aug = transforms_factory.create_transform(
    input_size=224,
    is_training=False
)

In [15]:
dm = ImageDataModule(
    task="classification",
    batch_size=32,
    # train
    train_image_paths=train_image_fpaths[1000:],
    train_augmentations=train_aug,
    train_targets=train_targets[1000:],
    # valid
    val_image_paths=train_image_fpaths[:1000],
    val_augmentations=val_aug,
    val_targets=train_targets[:1000],
    # test
    test_image_paths=train_image_fpaths,
    test_augmentations=val_aug
)
dm.setup()

In [16]:
from ml.learner import ImageClassifier
from ml.params import load_cfg

In [17]:
cfg = load_cfg(fpath="../params.yaml", cfg_name=f"train_two")
cfg.metric = "auc"
cfg.metric_mode = "max"
cfg.bs = 64
cfg

{'name': 'two', 'seed': 7591, 'n_folds': 5, 'fold': -1, 'metric': 'auc', 'metric_mode': 'max', 'train_data': 'data/train', 'arch': 'swin_large_patch4_window7_224', 'pretrained': True, 'epochs': 6, 'bs': 64, 'auto_batch_size': False, 'accumulate_grad_batches': 1, 'precision': 'bf16', 'use_normalize': True, 'n_tfms': 1, 'magn': 5, 'sz': 224, 'use_mix': 0, 'mix_p': 0.0, 'resize': -1, 'dropout': 0.0, 'wd': 0.0, 'label_smoothing': 0.1, 'loss': 'bce_with_logits', 'opt': 'adamw', 'sched': 'cosine', 'warmup_epochs': 1, 'lr': 2.4e-05, 'auto_lr': False, 'mom': 0.9}

In [18]:
model = ImageClassifier(
    in_channels=3,
    num_classes=1,
    cfg=cfg,
    pretrained=True
)

In [19]:
import pytorch_lightning as pl

In [20]:
trainer = pl.Trainer(
    gpus=1,
    precision=16,
    max_epochs=cfg.epochs,
)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [21]:
trainer.fit(model, dm)

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name         | Type            | Params
-------------------------------------------------
0 | backbone     | SwinTransformer | 194 M 
1 | head         | Sequential      | 8.3 K 
2 | train_metric | AUROC           | 0     
3 | val_metric   | AUROC           | 0     
-------------------------------------------------
195 M     Trainable params
0         Non-trainable params
195 M     Total params
390.008   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Epoch 3 // train metric: 0.7709, valid metric: 0.7674


Validating: 0it [00:00, ?it/s]

Epoch 4 // train metric: 0.8011, valid metric: 0.7698


Validating: 0it [00:00, ?it/s]

Epoch 5 // train metric: 0.8248, valid metric: 0.7705


In [22]:
preds = trainer.predict(model, dm.test_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


TrainerFn.PREDICTING


Predicting: 277it [00:00, ?it/s]

In [30]:
(preds > 0.4).sum()

415

In [31]:
np.array(train_targets).sum()

578

## Calibrate classifier

In [73]:
from sklearn.metrics import precision_score, recall_score, f1_score


thres = 0.31
print(f"TP+FP: {(preds > thres).sum()}")
print(f"Precision: {precision_score(y_true=train_targets, y_pred=(preds > thres)):.2%}")
print(f"Recall: {recall_score(y_true=train_targets, y_pred=(preds > thres)):.2%}")
print(f"F1: {f1_score(y_true=train_targets, y_pred=(preds > thres)):.2%}")

TP+FP: 603
Precision: 58.87%
Recall: 61.42%
F1: 60.12%


In [71]:
from sklearn.metrics import confusion_matrix

confusion_matrix(train_targets, (preds > thres))

array([[9049,  248],
       [ 223,  355]])

In [23]:
import numpy as np

preds = np.vstack(preds)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

nrows = 20
fig, axs = plt.subplots(nrows=nrows, ncols=1, figsize=(10, nrows*10))
for i, fpath in enumerate(test_image_fpaths[:nrows]):
    img = Image.open(fpath)
    axs[i].set_title(f"{'dog' if preds[i][0] > 0.5 else 'cat'}")
    axs[i].imshow(img)

In [ ]:
import pandas as pd

df = pd.read_csv("../data/train_5folds.csv")
df.head()

In [ ]:
feat = pd.DataFrame({
    "name": [p.stem for p in test_image_fpaths],
    "isdog": preds.squeeze().tolist()
})
feat = feat[feat.name.isin(df.Id)]
feat.head()

In [ ]:
df.shape[0] == feat.shape[0]

In [ ]:
feat = feat.sort_values("name").reset_index(drop=True)
feat["kfold"] = [int(v) for v in df["kfold"]]
feat.head()

In [ ]:
feat.to_csv("../data/dogvscat_feature.csv")